## Demonstration of the Item-Based Collaborative Recommender

This system leverages collaborative filtering by analyzing how user-item interactions bridges items
Therefore, it focuses on the user-item relation.

It recommends articles that these similar users have engaged with, aiming to provide personalized suggestions. The model's performance is evaluated using MAP@K and NDCG@K metrics.

In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

import polars as pl
import numpy as np

from parquet_data_reader import ParquetDataReader
from utils.data_preprocessing import DataProcesser
from models.collaborative.item_based_CF import ItemBasedCollaborativeRecommender

pl.Config.set_tbl_cols(-1)

polars.config.Config

## Data import & Preprocessing

In [2]:
data_reader = ParquetDataReader()
articles_df = data_reader.read_data('../../data/articles.parquet')
train_behaviors_df = data_reader.read_data('../../data/train/behaviors.parquet')
train_history_df = data_reader.read_data('../../data/train/history.parquet')
document_vectors_df = data_reader.read_data('../../data/document_vector.parquet')

We check the size of the data. From the sizes we learn that:
<ol>
  <li>We have 20738 unique articles</li>
  <li>We have 15143 unique users</li>
  <li>We have 232887 interactions in the testset</li>
</ol> 

In [3]:
print("articles_df has the size:         ", articles_df.shape)
print("train_behaviors_df has the size:  ", train_behaviors_df.shape)
print("train_history_df has the size:    ", train_history_df.shape)
print("document_vectors_df has the size: ", document_vectors_df.shape)

articles_df has the size:          (20738, 21)
train_behaviors_df has the size:   (232887, 17)
train_history_df has the size:     (15143, 5)
document_vectors_df has the size:  (125541, 2)


### Validation set

In [4]:
test_behaviours_df = data_reader.read_data('../../data/validation/behaviors.parquet')
test_behaviours_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
96791,null,2023-05-28 04:21:24,9.0,null,2,"[9783865, 9784591, … 9784710]",[9784696],22548,false,null,null,null,false,142,72.0,100.0
96798,null,2023-05-28 04:31:48,46.0,null,2,"[9782884, 9783865, … 9784648]",[9784281],22548,false,null,null,null,false,143,16.0,28.0
96801,null,2023-05-28 04:30:17,14.0,null,2,"[9784648, 7184889, … 9781983]",[9784444],22548,false,null,null,null,false,143,12.0,24.0
96808,null,2023-05-28 04:27:19,22.0,null,2,"[9784607, 9695098, … 9781983]",[9781983],22548,false,null,null,null,false,142,125.0,80.0
96810,null,2023-05-28 04:29:47,23.0,null,2,"[9781983, 7184889, … 9781520]",[9784642],22548,false,null,null,null,false,142,null,null


In [5]:
# Combine train and test behaviors
combined_df = pl.concat([train_behaviors_df, test_behaviours_df])

# Generate a random mask for splitting
n = combined_df.height  # Total number of rows
test_mask = np.random.rand(n) < 0.30  # 30% test, 70% train

# Apply the mask
test_behaviors_df = combined_df.filter(test_mask)
train_behaviors_df = combined_df.filter(~test_mask)

# Verify the split
print(f"Train size: {train_behaviors_df.shape[0]}, Test size: {test_behaviors_df.shape[0]}")

Train size: 334536, Test size: 142998


### Table contents

The information on news articles. As we are going to perform user-user CF, this table is not neccesary

In [6]:
articles_df.head()

article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str
3001353,"""Natascha var ikke den første""","""Politiet frygter nu, at Natasc…",2023-06-29 06:20:33,false,"""Sagen om den østriske Natascha…",2006-08-31 08:06:45,[3150850],"""article_default""","""https://ekstrabladet.dk/krimi/…",[],[],"[""Kriminalitet"", ""Personfarlig kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative"""
3003065,"""Kun Star Wars tjente mere""","""Biografgængerne strømmer ind f…",2023-06-29 06:20:35,false,"""Vatikanet har opfordret til at…",2006-05-21 16:57:00,[3006712],"""article_default""","""https://ekstrabladet.dk/underh…",[],[],"[""Underholdning"", ""Film og tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive"""
3012771,"""Morten Bruun fyret i Sønderjys…","""FODBOLD: Morten Bruun fyret me…",2023-06-29 06:20:39,false,"""Kemien mellem spillerne i Supe…",2006-05-01 14:28:40,[3177953],"""article_default""","""https://ekstrabladet.dk/sport/…",[],[],"[""Erhverv"", ""Kendt"", … ""Ansættelsesforhold""]",142,"[196, 199]","""sport""",null,null,null,0.8241,"""Negative"""
3023463,"""Luderne flytter på landet""","""I landets tyndest befolkede om…",2023-06-29 06:20:43,false,"""Det frække erhverv rykker på l…",2007-03-24 08:27:59,[3184029],"""article_default""","""https://ekstrabladet.dk/nyhede…",[],[],"[""Livsstil"", ""Erotik""]",118,[133],"""nyheder""",null,null,null,0.7053,"""Neutral"""
3032577,"""Cybersex: Hvornår er man utro?""","""En flirtende sms til den flott…",2023-06-29 06:20:46,false,"""De fleste af os mener, at et t…",2007-01-18 10:30:37,[3030463],"""article_default""","""https://ekstrabladet.dk/sex_og…",[],[],"[""Livsstil"", ""Partnerskab""]",565,[],"""sex_og_samliv""",null,null,null,0.9307,"""Neutral"""


Each file consists of seven days of impression logs. The train_behaviors_df table contains all information about interactions between users and items, and can be used as a basis for user-user CF. <strong>Therefore we only need this table</strong>.

In [7]:
train_behaviors_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
149474,null,2023-05-24 07:47:53,13.0,null,2,"[9778623, 9778682, … 9778728]",[9778657],139836,false,null,null,null,false,759,7.0,22.0
150528,null,2023-05-24 07:33:25,25.0,null,2,"[9778718, 9778728, … 9778682]",[9778623],143471,false,null,null,null,false,1240,287.0,100.0
153070,9777492,2023-05-24 07:13:14,26.0,100.0,1,"[9020783, 9778444, … 9778628]",[9778628],151570,false,null,null,null,false,1976,4.0,18.0
153071,9778623,2023-05-24 07:11:08,125.0,100.0,1,"[9777492, 9774568, … 9775990]",[9777492],151570,false,null,null,null,false,1976,26.0,100.0
153075,9777492,2023-05-24 07:13:58,26.0,100.0,1,"[9778500, 9776420, … 9020783]",[9777034],151570,false,null,null,null,false,1976,7.0,16.0


Each file consists of users' click histories collected over 21 days period. This table does contain the same values as the train_behaviours_df, but as that table is easier to work with we will use train_behaviours_df over this one

In [8]:
train_history_df.head()

user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
u32,list[datetime[μs]],list[f32],list[i32],list[f32]
13538,"[2023-04-27 10:17:43, 2023-04-27 10:18:01, … 2023-05-17 20:36:34]","[100.0, 35.0, … 100.0]","[9738663, 9738569, … 9769366]","[17.0, 12.0, … 16.0]"
14241,"[2023-04-27 09:40:18, 2023-04-27 09:40:33, … 2023-05-17 17:08:41]","[100.0, 46.0, … 100.0]","[9738557, 9738528, … 9767852]","[8.0, 9.0, … 12.0]"
20396,"[2023-04-27 12:30:44, 2023-04-27 12:31:34, … 2023-05-17 10:59:44]","[100.0, 59.0, … 13.0]","[9738760, 9738355, … 9769679]","[49.0, 34.0, … 4.0]"
34912,"[2023-04-29 07:12:49, 2023-04-29 13:01:18, … 2023-05-18 05:06:40]","[100.0, 35.0, … 27.0]","[9741802, 9741804, … 9770882]","[153.0, 7.0, … 5.0]"
37953,"[2023-04-27 19:17:10, 2023-04-27 19:17:27, … 2023-05-17 21:29:22]","[14.0, 28.0, … 18.0]","[9739205, 9739202, … 9769306]","[4.0, 16.0, … 6.0]"


List of vectors for each article. This is used to describe the items. It could be used for item-item CF, but is not relevant to user-user CF.  <strong>This table will therefore not be used</strong>

In [9]:
document_vectors_df.head()

article_id,document_vector
i32,list[f32]
3000022,"[0.065424, -0.047425, … 0.035706]"
3000063,"[0.028815, -0.000166, … 0.027167]"
3000613,"[0.037971, 0.033923, … 0.063961]"
3000700,"[0.046524, 0.002913, … 0.023423]"
3000840,"[0.014737, 0.024068, … 0.045991]"


From the analasys we see that we only need train_behaviour_df to perform user-user CF

## Preprocessing

### Remove non-needed values

We see that we have several items that are not required for performing user-user CF

In [10]:
train_behaviors_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
149474,null,2023-05-24 07:47:53,13.0,null,2,"[9778623, 9778682, … 9778728]",[9778657],139836,false,null,null,null,false,759,7.0,22.0
150528,null,2023-05-24 07:33:25,25.0,null,2,"[9778718, 9778728, … 9778682]",[9778623],143471,false,null,null,null,false,1240,287.0,100.0
153070,9777492,2023-05-24 07:13:14,26.0,100.0,1,"[9020783, 9778444, … 9778628]",[9778628],151570,false,null,null,null,false,1976,4.0,18.0
153071,9778623,2023-05-24 07:11:08,125.0,100.0,1,"[9777492, 9774568, … 9775990]",[9777492],151570,false,null,null,null,false,1976,26.0,100.0
153075,9777492,2023-05-24 07:13:58,26.0,100.0,1,"[9778500, 9776420, … 9020783]",[9777034],151570,false,null,null,null,false,1976,7.0,16.0


All information that does not describe a user, or a user-item interaction can therefore be removed

In [11]:
irelevant_columns = ["impression_time", "device_type", "article_ids_inview", "article_ids_clicked", "session_id", "next_read_time", "next_scroll_percentage"]
train_behaviors_df = train_behaviors_df.drop(irelevant_columns)
train_behaviors_df.head()

impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,i32,f32,f32,u32,bool,i8,i8,i8,bool
149474,null,13.0,null,139836,false,null,null,null,false
150528,null,25.0,null,143471,false,null,null,null,false
153070,9777492,26.0,100.0,151570,false,null,null,null,false
153071,9778623,125.0,100.0,151570,false,null,null,null,false
153075,9777492,26.0,100.0,151570,false,null,null,null,false


The remaining items are the ones that can be used. But already here we see that we have several features with lacking information. We should therefore treat this

### Account for missing values

We see here that alot of the behaviours contain missing values. The therefore have to either remove or replace the values

In [12]:
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(334536, 10)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,235415,0,236944,0,0,311709,328119,325708,0


In [13]:
train_behaviors_df = train_behaviors_df.filter(train_behaviors_df["article_id"].is_not_null())
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(99121, 10)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,2590,0,0,92342,96895,95756,0


We see that of 70421, are there between 65-68000 missing values for gender, postcode and age. We therefore remove these as there is no use subsidizing them

In [14]:
train_behaviors_df = train_behaviors_df.drop(["gender", "postcode", "age"])
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(99121, 7)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,is_subscriber
u32,u32,u32,u32,u32,u32,u32
0,0,0,2590,0,0,0


We still see that 2570/98967 rows are missing a scroll percentage. As this is very low (<3%) we can easily replace this. Intitially we just set scroll to 0

In [15]:
train_behaviors_df = train_behaviors_df.fill_null(strategy="zero")

### Account for multiple instances of the same article and user

By checking rows where the user_id and article_id are the same we see that we have 9855 instances where the user has read the same article multiple times

In [16]:
duplicates = train_behaviors_df.group_by(["article_id", "user_id"]).count().filter(pl.col("count") > 1)

print(duplicates)

shape: (12_424, 3)
┌────────────┬─────────┬───────┐
│ article_id ┆ user_id ┆ count │
│ ---        ┆ ---     ┆ ---   │
│ i32        ┆ u32     ┆ u32   │
╞════════════╪═════════╪═══════╡
│ 9780372    ┆ 2120961 ┆ 3     │
│ 9783774    ┆ 830242  ┆ 2     │
│ 9770792    ┆ 2581317 ┆ 2     │
│ 9780547    ┆ 1796372 ┆ 2     │
│ 9789810    ┆ 2049682 ┆ 3     │
│ …          ┆ …       ┆ …     │
│ 9772092    ┆ 2074079 ┆ 2     │
│ 9775562    ┆ 111082  ┆ 5     │
│ 9767697    ┆ 2572635 ┆ 2     │
│ 9785668    ┆ 1795891 ┆ 2     │
│ 9778718    ┆ 2379982 ┆ 2     │
└────────────┴─────────┴───────┘


C:\Users\magnu\AppData\Local\Temp\ipykernel_22292\4233841252.py:1: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  duplicates = train_behaviors_df.group_by(["article_id", "user_id"]).count().filter(pl.col("count") > 1)


We see that we need to combine these duplicate rows. We therefore propose that for multiple instances of the same article and user, we combine the readtime and select the largest scroll percentage. This way we can preserve the data without having duplicates

In [17]:
dataProcesser = DataProcesser()
behaviors_df = dataProcesser.collaborative_filtering_preprocess()
train_df, test_df = dataProcesser.random_split(behaviors_df, test_ratio=0.2)
print(train_df.head())

shape: (79_305, 4)
┌────────────┬─────────┬────────────────┬────────────┐
│ article_id ┆ user_id ┆ total_readtime ┆ max_scroll │
│ ---        ┆ ---     ┆ ---            ┆ ---        │
│ i32        ┆ u32     ┆ f32            ┆ f32        │
╞════════════╪═════════╪════════════════╪════════════╡
│ 9754160    ┆ 772964  ┆ 31.0           ┆ 100.0      │
│ 9770288    ┆ 2128426 ┆ 130.0          ┆ 100.0      │
│ 9783990    ┆ 1100577 ┆ 59.0           ┆ 100.0      │
│ 9770997    ┆ 1361367 ┆ 63.0           ┆ 100.0      │
│ 9775894    ┆ 1861113 ┆ 28.0           ┆ 100.0      │
│ …          ┆ …       ┆ …              ┆ …          │
│ 9785049    ┆ 1442573 ┆ 126.0          ┆ 100.0      │
│ 9771128    ┆ 80479   ┆ 1.46832e6      ┆ 100.0      │
│ 9790700    ┆ 1771982 ┆ 674.0          ┆ 100.0      │
│ 9788471    ┆ 1183758 ┆ 50.0           ┆ 100.0      │
│ 9781017    ┆ 2314174 ┆ 51.0           ┆ 100.0      │
└────────────┴─────────┴────────────────┴────────────┘


## Model Fit

This first model uses readtime and read percentage interactions to compare the user interactions 

In [18]:
recommender = ItemBasedCollaborativeRecommender(train_df)
recommender.fit()

{9754160: [(9788043, np.float64(0.6306408561911887)),
  (9778827, np.float64(0.4688516063397338)),
  (9766379, np.float64(0.31162769797518564)),
  (9705749, np.float64(0.26704122197873126)),
  (9782879, np.float64(0.18545002681605016)),
  (9779122, np.float64(0.17606447935320801)),
  (9776544, np.float64(0.06941049203624383)),
  (9754241, np.float64(0.06319456293555625)),
  (9777950, np.float64(0.023761076224129596)),
  (9773084, np.float64(0.018111472652194482))],
 9770288: [(9744530, np.float64(0.08266300611850053)),
  (9771779, np.float64(0.0826629523390392)),
  (9741415, np.float64(0.08266196844985862)),
  (9775809, np.float64(0.08094045077077439)),
  (9770450, np.float64(0.07213810090003403)),
  (9761859, np.float64(0.06921565928093765)),
  (9776449, np.float64(0.041950623776855656)),
  (9773172, np.float64(0.031214297775008948)),
  (9710762, np.float64(0.02346511779273064)),
  (9436758, np.float64(0.0020506293455943014))],
 9783990: [(9765943, np.float64(0.1542192496936473)),
  (

This first model just compares all artilces read by users when comparing users

In [19]:
binary_recommender = ItemBasedCollaborativeRecommender(train_df, binary_model=True)
binary_recommender.fit()

{9754160: [(9677792, np.float64(0.19245008972987532)),
  (9630091, np.float64(0.13608276348795434)),
  (9738980, np.float64(0.13608276348795434)),
  (9776332, np.float64(0.13608276348795434)),
  (9766379, np.float64(0.13608276348795434)),
  (9670069, np.float64(0.13608276348795434)),
  (9714262, np.float64(0.13608276348795434)),
  (9359995, np.float64(0.13608276348795434)),
  (9709095, np.float64(0.13608276348795434)),
  (9602118, np.float64(0.13608276348795434))],
 9770288: [(9766379, np.float64(0.11624763874381927)),
  (9716681, np.float64(0.11624763874381927)),
  (8934043, np.float64(0.11624763874381927)),
  (9744530, np.float64(0.11624763874381927)),
  (9749847, np.float64(0.11624763874381927)),
  (9762288, np.float64(0.11624763874381927)),
  (9701289, np.float64(0.11624763874381927)),
  (9775207, np.float64(0.11185934253567076)),
  (9775500, np.float64(0.09687303228651611)),
  (9775361, np.float64(0.09004503377814965))],
 9783990: [(9563703, np.float64(0.13245323570650436)),
  (97

Of the original 15143 users, only 9194 can be accounted for with the current solution. This should be changed in the future

## Model presentation

### Article Reccomendation

In [20]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9785596, 9777846, 9790293, 9778155, 9784852]
reccomended for user  620796 :  [9765326, 9786209, 9779365, 9767490, 9788126]
reccomended for user  1067393 :  [9782133, 9773341, 9773887, 9780925, 9788547]
reccomended for user  1726258 :  [9786209, 9778842, 9777324, 9771473, 9765326]
reccomended for user  17205 :  [9773296, 9788095, 9784702, 9779269, 9771846]


In [21]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", binary_recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9774864, 9672256, 9673979, 9738729, 9771627]
reccomended for user  620796 :  [9765326, 9749901, 9306867, 9678305, 9749469]
reccomended for user  1067393 :  [9639858, 9728471, 9767975, 9784679, 9788898]
reccomended for user  1726258 :  [9657797, 9765326, 9306867, 9500467, 9678305]
reccomended for user  17205 :  [9768638, 9766625, 9530940, 9482380, 9364014]


### Evaluation Scores

#### Without ability to reccomend read articles

The complex model only reccomending articles the user has not yet read

In [22]:
results = recommender.evaluate_recommender(test_df, k=10, n_jobs=4, user_sample=200, allow_read_articles=False)
results

{'MAP@K': np.float64(0.0011695906432748538),
 'NDCG@K': np.float64(0.0011841124682602704)}

The binary reccomender model only reccomending articles the user has not yet read

In [23]:
results = binary_recommender.evaluate_recommender(test_df, k=10, n_jobs=4, user_sample=200, allow_read_articles=False)
results

{'MAP@K': np.float64(0.0011494252873563218),
 'NDCG@K': np.float64(0.0014639265533243047)}

#### With ability to reccomend previously read articles

The complex model reccomending articles the user, even if they have read them before

In [24]:
results = recommender.evaluate_recommender(test_df, k=10, n_jobs=4, user_sample=200, allow_read_articles=True)
results

{'MAP@K': np.float64(0.012790697674418606),
 'NDCG@K': np.float64(0.02164330932436144)}

The binary reccomender model reccomending articles the user, even if they have read them before

In [25]:
results = binary_recommender.evaluate_recommender(test_df, k=10, n_jobs=4, user_sample=200, allow_read_articles=True)
results

{'MAP@K': np.float64(0.012048192771084338),
 'NDCG@K': np.float64(0.01720310106898417)}

## Model Experimentation

In [26]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=100, allow_read_articles=True)
results = set(test_behaviours_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9786243, 9787524, 9781902, 9784591, 9783824, 9786111, 9776916, 9779615, 9788705, 9789473, 9428643, 9783334, 9782315, 9756075, 9787441, 9782722, 9786821, 9782726, 9786566, 9789896, 9787465, 9788362, 9791049, 9782092, 9780815, None, 9783509, 9772508, 9786718, 9786719, 9787487, 9790942, 9783655, 9786351, 9780849, 9781875, 9788661, 9781878, 9787510, 9786618, 9673979, 9780348, 9781887}
[9785596, 9777846, 9790293, 9778155, 9784852, 9778413, 9787261, 9782180, 4265340, 9775985, 9774764, 9774864, 9780195, 9770997, 9789502, 8534547, 9488208, 9777299, 9771355, 9790548, 9672256, 9789404, 9780791, 9787846, 9788692, 9771065, 9785973, 9785076, 9772300, 9775763, 9773137, 9789664, 9777026, 9665320, 9659315, 9514605, 9790755, 9769743, 9781316, 9782468, 9779204, 9778813, 9772751, 9783405, 9785062, 9785986, 9789494, 9778219, 9673979, 9779242, 9783852, 9788552, 9785828, 9780193, 9749349, 9783977, 9594548, 9775804, 9766635, 9783057, 9776147, 9775518, 9711985, 9782308, 9782722, 9770045, 9776691, 9776694, 97

In [27]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=100, allow_read_articles=True)
results = set(test_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9786243, 9787524, 9781902, 9784591, 9783824, 9786111, 9776916, 9779615, 9788705, 9789473, 9428643, 9783334, 9782315, 9756075, 9787441, 9782722, 9786821, 9782726, 9786566, 9789896, 9787465, 9788362, 9791049, 9782092, 9780815, None, 9783509, 9772508, 9786718, 9786719, 9787487, 9790942, 9783655, 9786351, 9780849, 9781875, 9788661, 9781878, 9787510, 9786618, 9673979, 9780348, 9781887}
[9785596, 9777846, 9790293, 9778155, 9784852, 9778413, 9787261, 9782180, 4265340, 9775985, 9774764, 9774864, 9780195, 9770997, 9789502, 8534547, 9488208, 9777299, 9771355, 9790548, 9672256, 9789404, 9780791, 9787846, 9788692, 9771065, 9785973, 9785076, 9772300, 9775763, 9773137, 9789664, 9777026, 9665320, 9659315, 9514605, 9790755, 9769743, 9781316, 9782468, 9779204, 9778813, 9772751, 9783405, 9785062, 9785986, 9789494, 9778219, 9673979, 9779242, 9783852, 9788552, 9785828, 9780193, 9749349, 9783977, 9594548, 9775804, 9766635, 9783057, 9776147, 9775518, 9711985, 9782308, 9782722, 9770045, 9776691, 9776694, 97

In [12]:
from utils.evaluation import perform_model_evaluation
metrics = perform_model_evaluation(binary_recommender, test_data=test_df, k=5)
metrics

{'precision@k': np.float64(0.0012476837554045708),
 'recall@k': np.float64(0.0006524706559429079),
 'fpr@k': np.float64(0.0013696716999908412)}

In [29]:
from utils.evaluation import append_model_metrics
append_model_metrics(metrics, "item based")

### Carbon Footprint
This section creates an emissions.csv file in the "output"-folder
It utilizes the code_carbon (`codecarbon EmissionsTracker`) to record the carbon footprint of the `fit` and the `recommend` methods of the model.

In [14]:
from utils.evaluation import track_model_energy

print("\nCarbon footprint of the recommender:")
footprint = track_model_energy(recommender, "item_based", user_id=test_user_id, n=5)
footprint

[codecarbon INFO @ 11:01:49] [setup] RAM Tracking...
[codecarbon INFO @ 11:01:49] [setup] CPU Tracking...
[codecarbon WARNING @ 11:01:49] No CPU tracking mode found. Falling back on CPU constant mode. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU




Carbon footprint of the recommender:


[codecarbon WARNING @ 11:01:51] We saw that you have a 13th Gen Intel(R) Core(TM) i7-13700H but we don't know it. Please contact us.
[codecarbon INFO @ 11:01:51] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i7-13700H
[codecarbon INFO @ 11:01:51] [setup] GPU Tracking...
[codecarbon INFO @ 11:01:51] No GPU found.
[codecarbon INFO @ 11:01:51] >>> Tracker's metadata:
[codecarbon INFO @ 11:01:51]   Platform system: Windows-10-10.0.26100-SP0
[codecarbon INFO @ 11:01:51]   Python version: 3.11.9
[codecarbon INFO @ 11:01:51]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 11:01:51]   Available RAM : 15.731 GB
[codecarbon INFO @ 11:01:51]   CPU count: 20
[codecarbon INFO @ 11:01:51]   CPU model: 13th Gen Intel(R) Core(TM) i7-13700H
[codecarbon INFO @ 11:01:51]   GPU count: None
[codecarbon INFO @ 11:01:51]   GPU model: None
[codecarbon INFO @ 11:01:54] Saving emissions data to file c:\Users\magnu\NewDesk\An.sys\TDT4215\recommender_system\demostrations\output\item_based_fit_

{'fit': ({9754160: [(9788043, np.float64(0.6306408561911887)),
    (9778827, np.float64(0.4688516063397338)),
    (9766379, np.float64(0.31162769797518564)),
    (9705749, np.float64(0.26704122197873126)),
    (9782879, np.float64(0.18545002681605016)),
    (9779122, np.float64(0.17606447935320801)),
    (9776544, np.float64(0.06941049203624383)),
    (9754241, np.float64(0.06319456293555625)),
    (9777950, np.float64(0.023761076224129596)),
    (9773084, np.float64(0.018111472652194482))],
   9770288: [(9744530, np.float64(0.08266300611850053)),
    (9771779, np.float64(0.0826629523390392)),
    (9741415, np.float64(0.08266196844985862)),
    (9775809, np.float64(0.08094045077077439)),
    (9770450, np.float64(0.07213810090003403)),
    (9761859, np.float64(0.06921565928093765)),
    (9776449, np.float64(0.041950623776855656)),
    (9773172, np.float64(0.031214297775008948)),
    (9710762, np.float64(0.02346511779273064)),
    (9436758, np.float64(0.0020506293455943014))],
   9783990